# ResNet

In [6]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchsummary import summary
import renyan_utils as ry

## ResNet Module

In [9]:
class Residual(nn.Module):
    def __init__(self, in_channels, out_channels, use_lxlconv = False, stride = 1):
        super(Residual, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1, stride = stride)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1)
        if use_lxlconv:
            self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size = 1, stride = stride)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
    
    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return F.relu(Y + X)

In [14]:
blk = Residual(3, 3)
X = torch.rand((4, 3, 6, 6))
blk(X).shape

torch.Size([4, 3, 6, 6])

In [17]:
# half width & height
blk = Residual(3, 6, True, stride = 2)
blk(X).shape

torch.Size([4, 6, 3, 3])

## ResNet

In [26]:
net = nn.Sequential(nn.Conv2d(1, 64, kernel_size = 7, stride = 2, padding = 3),
                   nn.BatchNorm2d(64),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1))

In [27]:
def resnet_block(in_channels, out_channels, num_residuals, first_block = False):
    if first_block:
        assert in_channels == out_channels
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(in_channels, out_channels, use_lxlconv = True, stride = 2))
        else:
            blk.append(Residual(out_channels, out_channels))
        return nn.Sequential(*blk)

In [28]:
net.add_module("resnet_block1", resnet_block(64, 64, 2, first_block = True))
net.add_module("resnet_block2", resnet_block(64, 128, 2))
net.add_module("resnet_block3", resnet_block(128, 256, 2))
net.add_module("resnet_block4", resnet_block(256, 512, 2))

In [32]:
net.add_module("global_avg_pool", ry.GlobalAvgPool2d())
net.add_module("fc", nn.Sequential(ry.FlattenLayer(),
                                  nn.Linear(512, 10)))

In [35]:
X = torch.rand((1, 1, 256, 256))
for name, layer in net.named_children():
    X = layer(X)
    print(name, ' output shape:\t', X.shape)

0  output shape:	 torch.Size([1, 64, 128, 128])
1  output shape:	 torch.Size([1, 64, 128, 128])
2  output shape:	 torch.Size([1, 64, 128, 128])
3  output shape:	 torch.Size([1, 64, 64, 64])
resnet_block1  output shape:	 torch.Size([1, 64, 64, 64])
resnet_block2  output shape:	 torch.Size([1, 128, 32, 32])
resnet_block3  output shape:	 torch.Size([1, 256, 16, 16])
resnet_block4  output shape:	 torch.Size([1, 512, 8, 8])
global_avg_pool  output shape:	 torch.Size([1, 512, 1, 1])
fc  output shape:	 torch.Size([1, 10])


In [39]:
batch_size = 4
train_iter, test_iter = ry.load_data_fashion_mnist_resize(batch_size, resize = 96)

In [41]:
lr, num_epochs = 0.001, 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(net.parameters(), lr = lr)
ry.train_mnist_net(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)